In [1]:
import pandas as pd
import numpy as np
import datetime
import re
import json
import datetime
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Parameter Specification
date = None
cron_path = ''

In [4]:
# Parameters
date = "2020_03_14"


In [5]:
data = []
input_file = 'data/final_output/clean_data_{date}.json'.format(date=date)
input_file = os.path.join(cron_path,input_file)

with open(input_file) as data_file:
    for row in data_file:
        f = json.loads(row)
        data.append(f)

In [6]:
raw_df = pd.DataFrame(data)

In [7]:
# Building the actual df to work on

val_screen_vars = ['ticker','name','sector','industry','price','pe','pb','ps','pfcf','dividend_pct','evebitda','perf_year','perf_half_y','perf_quarter','perf_month']
df = pd.DataFrame()

for i in val_screen_vars:
    df[i] = raw_df[i]

In [8]:
""" All the data cleaning """

# Only look at tickers wher we have complete data
df = df[df.pb.notnull()]
df = df[df.ps.notnull()]
df = df[df.pfcf.notnull()]
df = df[df.evebitda.notnull()]

# Convert EVEBITDA to float
df['evebitda'] = df['evebitda'].apply(lambda x: float(str(x).replace(',','')))

# For div_pct and P/E, we can use null tickers
df['dividend_pct'] = df['dividend_pct'].fillna(0)
df['pe'] = df['pe'].fillna(1000)

# Dispose of industries we don't care about
dispose_df = df[(df['industry'].str.contains('REIT')) 
   | (df['industry'].isin(['USA','United Kingdom']))
   | (df['industry'].str.contains('Fund'))]

df = pd.concat([df, dispose_df]).drop_duplicates(keep=False)


In [9]:
# Assign master scores

for v in ['pe','pb','ps','pfcf','evebitda']:
    df[v+'_master_score'] = df[v].rank(pct=True,ascending=False)
    df[v+'_master_score'] = df[v+'_master_score'].apply(lambda x: int(x * 100))

df['dividend_pct_master_score'] = df['dividend_pct'].rank(pct=True,ascending=True)
df['dividend_pct_master_score'] = df['dividend_pct_master_score'].apply(lambda x: int(x * 100))

In [10]:
# Assign sector scores

for v in ['pe','pb','ps','pfcf','evebitda']:
    df[v+'_sector_score'] = df.groupby('sector')[v].rank(pct=True,ascending=False)
    df[v+'_sector_score'] = df[v+'_sector_score'].apply(lambda x: int(x * 100))

df['dividend_pct_sector_score'] = df.groupby('sector')['dividend_pct'].rank(pct=True,ascending=True)
df['dividend_pct_sector_score'] = df['dividend_pct_sector_score'].apply(lambda x: int(x * 100))

In [11]:
# Assign net scores

for l in ['_master_score','_sector_score']:
    df['net'+l] = df['pe'+l] + df['pb'+l] + df['pb'+l] + df['pfcf'+l]+ df['evebitda'+l] + df['dividend_pct'+l]

df['net_total_score'] = df[['net_master_score','net_sector_score']].astype(float).mean(axis=1)

In [12]:
# Normalize performance percentages

timeframes = ['year','half_y','quarter','month']

for t in timeframes:
    col = 'perf_' + t
    df[col + '_t_w'] = (df[col]-df[col].min())/(df[col].max()-df[col].min())
    
    sdf = df.groupby('sector')[col]
    df[col + '_s_w'] = (df[col]-sdf.transform(min))/(sdf.transform(max)-sdf.transform(min))
    
    if t == 'half_y':
        df[col + '_t_w'] = df[col + '_t_w'].apply(lambda x: x * 2)
        df[col + '_s_w'] = df[col + '_s_w'].apply(lambda x: x * 2)
    elif t == 'quarter':
        df[col + '_t_w'] = df[col + '_t_w'].apply(lambda x: x * 3)
        df[col + '_s_w'] = df[col + '_s_w'].apply(lambda x: x * 3)
    elif t == 'month':
        df[col + '_t_w'] = df[col + '_t_w'].apply(lambda x: x * 4)
        df[col + '_s_w'] = df[col + '_s_w'].apply(lambda x: x * 4)

df['perf_total_score'] = df['perf_year_t_w'] + df['perf_half_y_t_w'] + df['perf_quarter_t_w'] + df['perf_month_t_w']
df['perf_sector_score'] = df['perf_year_s_w'] + df['perf_half_y_s_w'] + df['perf_quarter_s_w'] + df['perf_month_s_w']

In [13]:
# Produce the DF with the key metrics and scores

final_df = df.filter(['ticker','name','sector','industry','price','pe','pb','ps','pfcf','dividend_pct','evebitda',
          'perf_year','perf_half_y','perf_quarter','perf_month','net_master_score','net_sector_score',
          'perf_total_score','perf_sector_score'])

# Produce the DF with top 25 stocks per sector

output_df = final_df.sort_values(by='net_sector_score', ascending=False).groupby('sector').head(25)

In [14]:
# Write files to CSV

scored_file = os.path.join(cron_path,'analysis_output/scored_file_{date}.csv'.format(date=date))
ranked_file = os.path.join(cron_path,'analysis_output/ranked_file_{date}.csv'.format(date=date))

final_df.to_csv(scored_file, index=False)
output_df.sort_values(
    by=['sector','perf_sector_score'],
    ascending=[True,False]
).to_csv(ranked_file, index=True)